# Unlearning DualTeacher


## 1. Setup e Import

In [1]:
!pip install rouge-score
import torch
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from rouge_score import rouge_scorer

# Configurazioni
MODEL_PATH = "/kaggle/input/olmo-model/semeval25-unlearning-1B-model"
DATA_PATH = "/kaggle/input/olmo-model/semeval25-unlearning-data"
MIA_VAL_PATH  = "/kaggle/input/mia-dataset-val"
MIA_TRAIN_PATH  = "/kaggle/input/mia-dataset"
GOOD_TEACHER_PATH = "/kaggle/input/good-teacher"

print(f"GPUs disponibili: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e47f1035cd16c4addde6d0eb205c42b738eb3b9aa482003d10d8f1276e8fad9a
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


2025-08-28 13:14:02.784761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756386843.124822      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756386843.232129      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPUs disponibili: 2
GPU 0: Tesla T4
GPU 1: Tesla T4


## 2. Caricamento Dati e Modelli

In [6]:
# Caricamento dataset
retain_train_df = pd.read_parquet(f"{DATA_PATH}/data/retain_train-00000-of-00001.parquet", engine='pyarrow')
retain_validation_df = pd.read_parquet(f"{DATA_PATH}/data/retain_validation-00000-of-00001.parquet", engine='pyarrow')
forget_train_df = pd.read_parquet(f"{DATA_PATH}/data/forget_train-00000-of-00001.parquet", engine='pyarrow')
forget_validation_df = pd.read_parquet(f"{DATA_PATH}/data/forget_validation-00000-of-00001.parquet", engine='pyarrow')

# Salvataggio in formato JSONL
!mkdir -p train validation
retain_train_df.to_json('train/retain.jsonl', orient='records', lines=True)
forget_train_df.to_json('train/forget.jsonl', orient='records', lines=True)
retain_validation_df.to_json('validation/retain.jsonl', orient='records', lines=True)
forget_validation_df.to_json('validation/forget.jsonl', orient='records', lines=True)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-0724-hf")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Dataset salvati e tokenizer caricato")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Dataset salvati e tokenizer caricato


## 3. Dataset

In [8]:
class UnlearningDataset(Dataset):
    def __init__(self, data_source, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        if isinstance(data_source, pd.DataFrame):
            self.data = data_source
            print(f"Caricati {len(self.data)} esempi dal DataFrame")
        elif isinstance(data_source, str):
            data_list = []
            with open(data_source, 'r', encoding='utf-8') as f:
                for line in f:
                    item = json.loads(line.strip())
                    data_list.append(item)
            self.data = pd.DataFrame(data_list)
            print(f"Caricati {len(self.data)} esempi da {data_source}")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        input_text = item["input"]
        output_text = item["output"]

        # Tokenizzazione unica
        combined = f"{input_text} {output_text}"
        tokenized = self.tokenizer(
            combined,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        # Tokenizzazione solo dell'input per start_locs
        input_ids = self.tokenizer(
            input_text,
            return_tensors="pt"
        )["input_ids"].squeeze(0)

        return {
            "input_ids": tokenized["input_ids"].squeeze(0),
            "attention_mask": tokenized["attention_mask"].squeeze(0),
            "start_locs": input_ids.size(0),  # posizione dove finisce l'input
            "labels": tokenized["input_ids"].squeeze(0),
            "split": 1 if item.get("split", "retain") == "forget" else 0
        }


In [9]:
# Create dataset and dataloader
batch_size = 4
train_data = pd.concat([retain_train_df, forget_train_df], ignore_index=True)
dataset = UnlearningDataset(train_data, tokenizer)
dataloader = DataLoader(dataset, batch_size, shuffle=True)

print(f"Dataset creato con {len(dataset)} esempi")

Caricati 2248 esempi dal DataFrame
Dataset creato con 2248 esempi


## 4. DualTeacher Trainer Class

In [ ]:
class DualTeacherTrainer:
    def __init__(self, model_path, tokenizer, teacher_lora_config, student_lora_config, device_map=None):
        self.model_path = model_path
        self.tokenizer = tokenizer
        self.teacher_lora_config = teacher_lora_config
        self.student_lora_config = student_lora_config
        self.device_map = device_map or {"student": "cuda:0", "teacher": "cuda:1"}
        
        self.good_teacher = None
        self.student_model = None
        self.initial_state_dict = {}
        
        
    def setup_models(self,skip_teacher_setup=False):
        """Initialize and setup both teacher and student models"""
        print("🔧 Setting up models...")
        base_model = AutoModelForCausalLM.from_pretrained(self.model_path, local_files_only=True)
        if skip_teacher_setup is False:
            # Setup good teacher with LoRA (for training)
            
            self.good_teacher = get_peft_model(base_model, self.teacher_lora_config, local_files_only=True)
            self.good_teacher = self.good_teacher.to(self.device_map["teacher"])
            self.good_teacher.print_trainable_parameters()
            
        # Setup student model with LoRA
        self.student_model = get_peft_model(base_model, self.student_lora_config)
        self.student_model = self.student_model.to(self.device_map["student"])
        self.student_model.print_trainable_parameters()
        
        # Save initial state for task vector calculation
        for name, param in self.student_model.named_parameters():
            if param.requires_grad:
                self.initial_state_dict[name] = param.data.clone()
        
        print("✅ Models setup completed")
        
    
    def create_bad_teacher_logits(self, good_teacher_logits: torch.Tensor):
        """Create more realistic bad teacher logits"""
        # Invece di -logits * 2.0, usa uniform distribution
        vocab_size = good_teacher_logits.size(-1)
        uniform_logits = torch.ones_like(good_teacher_logits) / vocab_size
        return uniform_logits + torch.randn_like(good_teacher_logits) * 0.1

        
    
    def compute_kl_divergence(self, batch):
        # Devices
        student_device = self.device_map["student"]
        teacher_device = self.device_map["teacher"]
    
        # Inputs
        input_ids_student = batch["input_ids"].to(student_device)
        attention_mask_student = batch["attention_mask"].to(student_device)
        labels_student = batch["labels"].to(student_device)
        split = batch["split"].float().to(student_device)
    
        # Student forward
        student_logits = self.student_model(input_ids_student, attention_mask=attention_mask_student).logits
        student_log_probs = torch.nn.functional.log_softmax(student_logits, dim=-1).to(teacher_device)
    
        # Teacher forward (no grad)
        input_ids_teacher = batch["input_ids"].to(teacher_device)
        attention_mask_teacher = batch["attention_mask"].to(teacher_device)
    
        with torch.no_grad():
            good_teacher_logits = self.good_teacher(input_ids_teacher, attention_mask=attention_mask_teacher).logits
            bad_teacher_logits = self.create_bad_teacher_logits(good_teacher_logits)
            good_teacher_probs = torch.nn.functional.softmax(good_teacher_logits, dim=-1)
            bad_teacher_probs = torch.nn.functional.softmax(bad_teacher_logits, dim=-1)
    
        # Masks retain/forget
        retain_mask = (split <= 0.5).to(teacher_device)
        forget_mask = (split > 0.5).to(teacher_device)
    
        total_loss = 0.0
        if retain_mask.any():
            retain_kl = torch.nn.functional.kl_div(
                student_log_probs[retain_mask],
                good_teacher_probs[retain_mask.bool()],
                reduction="none",
                log_target=False
            ).sum(dim=-1)  # somma su vocab
            retain_kl = retain_kl.mean()
            total_loss += 1.5 * retain_kl  # retain_weight
    
        if forget_mask.any():
            forget_kl = torch.nn.functional.kl_div(
                student_log_probs[forget_mask],
                bad_teacher_probs[forget_mask.bool()],
                reduction="none",
                log_target=False
            ).sum(dim=-1)
            forget_kl = forget_kl.mean()
            total_loss += 5.0 * forget_kl  # forget_weight
    
        # Entropia student per regolarizzazione
        entropy_loss = -(student_log_probs.exp() * student_log_probs).sum(-1).mean()
    
        return total_loss + 0.2 * entropy_loss

    
        
    def train_good_teacher(self, dataloader, num_epochs=2, lr=1e-4, save_path="good_teacher_adapter"):
        """Train the good teacher on retain samples using LoRA"""
        print("🚀 Training good teacher with LoRA...")

        self.good_teacher.to(self.device_map["teacher"])
        self.good_teacher.train()
        optimizer = torch.optim.AdamW(self.good_teacher.parameters(), lr=lr)
        
        for epoch in range(num_epochs):
            print(f"📅 Epoca {epoch + 1}/{num_epochs} - Good Teacher Training")
            
            epoch_losses = []
            retain_batches_processed = 0
            
            with tqdm(total=len(dataloader), desc=f"Good Teacher Epoca {epoch+1}") as pbar:
                for batch in dataloader:
                    # Filter retain samples only
                    split = batch['split']
                    retain_mask = (split == 0)
                    
                    if not retain_mask.any():
                        pbar.update(1)
                        continue
                    
                    # Extract retain samples
                    input_ids = batch['input_ids'][retain_mask].to(self.device_map["teacher"])
                    attention_mask = batch['attention_mask'][retain_mask].to(self.device_map["teacher"])
                    labels = batch['labels'][retain_mask].to(self.device_map["teacher"])
                    
                    if input_ids.size(0) == 0:
                        pbar.update(1)
                        continue
                    
                    optimizer.zero_grad()
                    outputs = self.good_teacher(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    
                    loss.backward()
                    optimizer.step()
                    
                    epoch_losses.append(loss.item())
                    retain_batches_processed += 1
                    pbar.update(1)
                    
                    if retain_batches_processed % 100 == 0:
                        pbar.set_postfix({"Loss": f"{loss.item():.4f}"})
            
            if epoch_losses:
                avg_loss = np.mean(epoch_losses)
                print(f"📊 Good Teacher Epoca {epoch+1} - Loss medio: {avg_loss:.4f}")
        
        print("✅ Good teacher training completed")
        
        # Save only LoRA adapter
        self.save_good_teacher(save_path)
        print(f"💾 Good teacher adapter salvato in {save_path}")
        
        
    def train_student(self, dataloader, num_epochs=4, lr=1e-4):
        """Train student model with dual teacher approach (student uses LoRA, teacher is base model)"""
        print("🚀 Training student with LoRA against base model teacher...")
        
        self.student_model.train()
        optimizer = torch.optim.AdamW(self.student_model.parameters(), lr=lr, weight_decay=0.01)
        
        for epoch in range(num_epochs):
            epoch_losses = []
            
            with tqdm(total=len(dataloader), desc=f"Student Epoca {epoch+1}") as pbar:
                for batch in dataloader:
                    optimizer.zero_grad()
                    loss = self.compute_kl_divergence(batch)
                    loss.backward()
                    optimizer.step()
                    
                    epoch_losses.append(loss.item())
                    pbar.set_postfix({"Loss": f"{loss.item():.4f}"})
                    pbar.update(1)
            
            avg_loss = np.mean(epoch_losses)
            print(f"Epoch {epoch+1} finished. Average Loss: {avg_loss:.4f}")
            
            # Save model after each epoch
            self.save_model(f"studentmodel_epoch_{epoch+1}")
        
        print("✅ Student training completed")
        
    def save_model(self, save_path):
        """Save student model and tokenizer"""
        self.student_model.save_pretrained(save_path)
        self.tokenizer.save_pretrained(save_path)
        
    def save_good_teacher(self, save_path):
        """Save only the LoRA adapter of the good teacher"""
        self.good_teacher.save_pretrained(save_path)


    def load_good_teacher_from_adapter(self, adapter_path):
        """Reload good teacher from base model + adapter"""
        print(f"📂 Caricamento good teacher da adapter {adapter_path} ...")
        base_model = AutoModelForCausalLM.from_pretrained(self.model_path, local_files_only=True)
        self.good_teacher = PeftModel.from_pretrained(base_model, adapter_path)
        self.good_teacher = self.good_teacher.to(self.device_map["teacher"])
        
        # Freeze params
        self.good_teacher.eval()
        for param in self.good_teacher.parameters():
            param.requires_grad = False
        print("✅ Good teacher caricato e congelato")

    def load_teacher(self,GOOD_TEACHER_PATH):
        self.good_teacher = AutoModelForCausalLM.from_pretrained(GOOD_TEACHER_PATH)
        self.good_teacher.eval()  # congelalo
        for param in self.good_teacher.parameters():
            param.requires_grad = False
        self.good_teacher.to(trainer.device_map["teacher"])
        print("✅ Good teacher caricato e congelato")

    
    def calculate_task_vector(self):
        """Calculate task vector from initial to final state"""
        task_vector = {}
        for name, param in self.student_model.named_parameters():
            if param.requires_grad and name in self.initial_state_dict:
                task_vector[name] = param.data - self.initial_state_dict[name]
        return task_vector

## 5. Setup Trainer and Training

In [ ]:
train_good_teacher = False

In [ ]:
# Configure LoRA for teacher and student
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    bias="none",
)


# Initialize trainer
trainer = DualTeacherTrainer(
    model_path=MODEL_PATH,
    tokenizer=tokenizer,
    teacher_lora_config=lora_config,
    student_lora_config=lora_config,
    device_map={"student": "cuda:0", "teacher": "cuda:1"}
)
# # Setup models

if train_good_teacher:
    trainer.setup_models()
    # Train good teacher (will be converted to base model after training)
    trainer.train_good_teacher(dataloader, num_epochs=5, lr=5e-5) 
    

In [ ]:
# Setup dei modelli (student viene comunque inizializzato)
trainer.setup_models(skip_teacher_setup=True)

trainer.load_good_teacher_from_adapter(GOOD_TEACHER_PATH)
# trainer.load_teacher(GOOD_TEACHER_PATH)

# Ora puoi allenare direttamente lo student
trainer.train_student(dataloader, num_epochs=5, lr=3e-5)
trainer.save_model("studentmodel_final")

## 6. Evaluation

In [ ]:

import types
import random, torch, numpy as np

try:
    import evaluation
    import importlib
    importlib.reload(evaluation)
except ImportError:
    pass

def run_evaluation(
    data_path,
    checkpoint_path,
    output_dir="eval_results",
    mia_data_path=MIA_TRAIN_PATH,
    mia_data_val_path=MIA_VAL_PATH,
    mmlu_metrics_file_path=None,
    max_new_tokens=256,
    batch_size=25,
    debug=False,
    compute_metrics_only=False,
    seed=42,
    keep_files=False,
):
    try:
        # Costruiamo un oggetto args simile a quello di argparse
        args = types.SimpleNamespace(
            data_path=data_path,
            checkpoint_path=checkpoint_path,
            output_dir=output_dir,
            mia_data_path=mia_data_path,
            mia_data_val_path=mia_data_val_path,
            mmlu_metrics_file_path=mmlu_metrics_file_path,
            max_new_tokens=max_new_tokens,
            batch_size=batch_size,
            debug=debug,
            compute_metrics_only=compute_metrics_only,
            seed=seed,
            keep_files=keep_files,
        )

        # Verifica che i file esistano
        print(f"🔍 Verificando paths...")
        print(f"  Data path: {data_path}")
        print(f"  Checkpoint path: {checkpoint_path}")
        print(f"  Output dir: {output_dir}")
        
        if not os.path.exists(data_path):
            raise FileNotFoundError(f"Data path not found: {data_path}")
        if not os.path.exists(checkpoint_path):
            raise FileNotFoundError(f"Checkpoint path not found: {checkpoint_path}")
        if not os.path.exists(os.path.join(data_path, 'forget.jsonl')):
            raise FileNotFoundError(f"forget.jsonl not found in {data_path}")
        if not os.path.exists(os.path.join(data_path, 'retain.jsonl')):
            raise FileNotFoundError(f"retain.jsonl not found in {data_path}")

        # Normalizza i path (come nello script originale)
        from pathlib import Path
        if args.output_dir is None:
            args.output_dir = os.getcwd()
        else:
            args.output_dir = args.output_dir.rstrip('/')
            Path(args.output_dir).mkdir(parents=True, exist_ok=True)

        # Lancia direttamente le funzioni
        random.seed(args.seed)
        torch.manual_seed(args.seed)
        np.random.seed(args.seed)

        from accelerate import Accelerator
        accelerator = Accelerator()

        if not args.compute_metrics_only:
            from transformers import AutoModelForCausalLM, AutoTokenizer
            from peft import PeftModel
            
            print(f"📥 Loading model from {args.checkpoint_path}...")
            
            # Carica il modello PEFT (LoRA) se è salvato come tale
            try:
                # Prima prova a caricare come modello PEFT
                base_model_path = MODEL_PATH  # Usa il path del modello base
                base_model = AutoModelForCausalLM.from_pretrained(
                    base_model_path, 
                    local_files_only=True,
                    torch_dtype=torch.bfloat16
                )
                model = PeftModel.from_pretrained(base_model, args.checkpoint_path)
                print("✅ Loaded as PEFT model")
            except:  
                # Se fallisce, prova a caricare come modello normale
                model = AutoModelForCausalLM.from_pretrained(
                    args.checkpoint_path,
                    torch_dtype=torch.bfloat16,
                    trust_remote_code=True
                )
                print("✅ Loaded as regular model")
            
            tokenizer = AutoTokenizer.from_pretrained(args.checkpoint_path)
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token

            print("🚀 Starting inference...")
            evaluation.inference(args, model, tokenizer)
            
            # if args.mia_data_path is not None:
                # print("🔍 Starting MIA attacks...")
                # evaluation.mia_attacks(args, model, tokenizer)

        if accelerator.is_main_process:
            print("📊 Computing metrics...")
            evaluation.compute_metrics(args)
            print("✅ Evaluation completed!")

    except Exception as e:
        print(f"❌ Error during evaluation: {e}")
        import traceback
        traceback.print_exc()

# === Step 4: Esegui evaluation ===
print("🎯 Starting evaluation process...")

# Verifica che i file esistano prima di iniziare
if os.path.exists("validation/forget.jsonl") and os.path.exists("validation/retain.jsonl"):
    if os.path.exists("studentmodel_final/"):
        run_evaluation(
            data_path="validation/",  # cartella relativa con forget.jsonl e retain.jsonl
            checkpoint_path="studentmodel_final/",  # cartella relativa con i pesi del modello
            output_dir="eval_results",
            debug=True  # Attiva debug per vedere cosa succede
        )
    else:
        print("❌ Model checkpoint not found at balanced_results/balanced_model/")
        print("   Make sure the training completed successfully")
else:
    print("❌ Validation files not found")
    print("   Expected: validation/forget.jsonl and validation/retain.jsonl")
    print("   Make sure the data processing completed successfully")

In [10]:
# import os
# import json
# import glob
# import pandas as pd
# import numpy as np
# import shutil
# from statistics import mean, harmonic_mean
# from rouge_score import rouge_scorer
# from sklearn.metrics import roc_curve, auc
# from pathlib import Path

# # Configurazione paths
# RESULTS_PATH = "/kaggle/input/eval-results"
# MIA_TRAIN_PATH = "/kaggle/input/mia-score/"
# OUTPUT_DIR = "final_metrics"

# def compute_nll(text: str):
#     enc = tokenizer(text, return_tensors="pt").to("cuda:0")
#     with torch.no_grad():
#         outputs = student_model(**enc, labels=enc["input_ids"])
#         # loss già è la NLL media per token
#         nll = outputs.loss.item()
#     return nll

# # === Funzione per caricare JSONL in DataFrame con NLL ===
# def load_jsonl_with_nll(path, label):
#     records = []
#     with open(path, "r") as f:
#         for line in f:
#             obj = json.loads(line)
#             text = obj.get("document", "")  # <-- puoi cambiare in "sentence_completion_task"]["input"]
#             nll = compute_nll(text)
#             records.append({
#                 "id": obj["id"],
#                 "text": text,
#                 "nll": nll,
#                 "label": label   # 1 = member, 0 = nonmember
#             })
#     return pd.DataFrame(records)

# def compute_auc(member_loss, nonmember_loss):
#     """Calcola AUC per MIA attack"""
#     assert not np.any(np.isnan(member_loss))
#     assert not np.any(np.isnan(nonmember_loss))
#     combined_loss = member_loss + nonmember_loss 
#     combined_loss = -1 * np.array(combined_loss)
#     combined_labels = len(member_loss) * [1] + len(nonmember_loss) * [0]
#     fp, tp, _ = roc_curve(combined_labels, combined_loss)
#     auc_score = float(auc(fp, tp))
#     return auc_score

# def compute_metrics():
#     """Calcola le metriche dai file CSV esistenti"""
#     print("📊 Iniziando calcolo metriche...")
    
#     # Crea directory output
#     Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    
#     scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
#     results = {}
#     aggregate_scores_list = []
    
#     # Processa forget e retain sets
#     for split in ['forget', 'retain']:
#         print(f"🔍 Processando {split} set...")
        
#         # Cerca file CSV per questo split
#         pattern = f"{RESULTS_PATH}/{split}_*.csv"
#         files = glob.glob(pattern)
        
#         if len(files) == 0:
#             print(f"❌ Nessun file trovato per pattern: {pattern}")
#             continue
        
#         print(f"   Trovati {len(files)} file: {files}")
        
#         # Carica e concatena tutti i CSV
#         df_list = []
#         for f in files:
#             try:
#                 df_temp = pd.read_csv(f)
#                 df_list.append(df_temp)
#                 print(f"   Caricato {f}: {len(df_temp)} righe")
#             except Exception as e:
#                 print(f"   ❌ Errore caricando {f}: {e}")
        
#         if not df_list:
#             print(f"❌ Nessun file valido per {split}")
#             continue
            
#         df = pd.concat(df_list, ignore_index=True)
#         print(f"   Dataset combinato: {len(df)} righe")
        
#         # Inizializza colonne per le metriche
#         df['regurgitation-score-rouge-1'] = None
#         df['regurgitation-score'] = None
#         df['knowledge-score'] = None
        
#         ground_truths = df['expected_output'].tolist()
#         gen_outputs = df['model_output'].tolist()
        
#         print(f"   Calcolando metriche per {len(ground_truths)} esempi...")
        
#         # Calcola metriche per ogni esempio
#         for i, (gen, gt) in enumerate(zip(gen_outputs, ground_truths)):
#             if pd.isna(df.loc[i, 'id']):
#                 continue
                
#             example_id = str(df.loc[i, 'id'])
            
#             # Task di regurgitation (string completion)
#             if example_id.endswith('sc') or 'sc' in example_id:
#                 try:
#                     rouge_scores = scorer.score(str(gt), str(gen))
#                     df.loc[i, 'regurgitation-score-rouge-1'] = rouge_scores['rouge1'].recall
#                     df.loc[i, 'regurgitation-score'] = rouge_scores['rougeL'].recall
#                 except Exception as e:
#                     print(f"   ⚠️ Errore ROUGE per esempio {i}: {e}")
                    
#             # Task di knowledge (question answering)  
#             elif example_id.endswith('qa') or 'qa' in example_id:
#                 try:
#                     is_correct = int(str(gt).strip().lower() == str(gen).strip().lower())
#                     df.loc[i, 'knowledge-score'] = is_correct
#                 except Exception as e:
#                     print(f"   ⚠️ Errore QA per esempio {i}: {e}")
        
#         # Converti None in NaN per il calcolo
#         df['regurgitation-score'] = pd.to_numeric(df['regurgitation-score'], errors='coerce')
#         df['knowledge-score'] = pd.to_numeric(df['knowledge-score'], errors='coerce')
        
#         # Calcola metriche aggregate per questo split
#         regurg_mean = np.nanmean(df['regurgitation-score'])
#         knowledge_mean = np.nanmean(df['knowledge-score'])
        
#         print(f"   Regurgitation score medio: {regurg_mean:.4f}")
#         print(f"   Knowledge score medio: {knowledge_mean:.4f}")
        
#         results[split+'-set'] = {
#             'overall-regurgitation-score': regurg_mean, 
#             'overall-knowledge-score': knowledge_mean
#         }
        
#         # Metriche per task (con gestione NaN)
#         task_groups = df.groupby('task')[['regurgitation-score', 'knowledge-score']]
#         split_aggregate_scores_dict = {}
        
#         for task, group in task_groups:
#             task_scores = {}
#             regurg_vals = pd.to_numeric(group['regurgitation-score'], errors='coerce')
#             knowledge_vals = pd.to_numeric(group['knowledge-score'], errors='coerce')
            
#             if not regurg_vals.isna().all():
#                 task_scores['regurgitation-score'] = np.nanmean(regurg_vals)
#             if not knowledge_vals.isna().all():
#                 task_scores['knowledge-score'] = np.nanmean(knowledge_vals)
                
#             split_aggregate_scores_dict[task] = task_scores
        
#         results[split+'-set'].update(split_aggregate_scores_dict)
        
#         # Aggrega scores per la metrica finale
#         split_aggregate_score_values = [float(val) for inner in split_aggregate_scores_dict.values() for val in inner.values() if not np.isnan(val)]
        
#         # Per forget set, inverti i valori (vogliamo "dimenticare")
#         if split == 'forget':
#             split_aggregate_score_values = [(1 - val) for val in split_aggregate_score_values]
        
#         aggregate_scores_list.extend(split_aggregate_score_values)
        
#         print(f"   ✅ {split} set completato")
    
#     # Processa MIA se disponibile
#     print("🔍 Processando MIA data...")
#     mia_files_found = False
    
#     for dataset in ['member', 'nonmember']:
#         pattern = f"{MIA_TRAIN_PATH}{dataset}.jsonl"
#         print(pattern)
#         files = glob.glob(pattern)
        
#         if files:
#             mia_files_found = True
#             break
    
#     if mia_files_found:
#         mia_results = {}
#         for dataset in ['member', 'nonmember']:
#             pattern = f"{MIA_TRAIN_PATH}{dataset}.jsonl"
#             print("pattern")
#             print(pattern)
#             files = glob.glob(pattern)
            
#             if files:
#                 print(f"   Trovati file {dataset}: {files}")
#                 records = []
#                 for f in files:
#                     df_raw = pd.read_json(f, lines=True)
#                     for _, row in df_raw.iterrows():
#                         text = row.get("document", "")  # oppure sentence_completion_task["input"]
#                         nll = compute_nll(text)         # <--- funzione che abbiamo definito prima
#                         records.append(nll)
#                 mia_results[dataset] = records
#                 print(f"   {dataset}: {len(mia_results[dataset])} esempi")
    
#         if 'member' in mia_results and 'nonmember' in mia_results:
#             auc = compute_auc(mia_results['member'], mia_results['nonmember'])
#             results['mia_loss_acc'] = auc
#             print(f"   ✅ MIA AUC: {auc:.4f}")
#         else:
#             print("   ❌ Dati MIA incompleti")
#     else:
#         print("   ❌ Nessun file MIA trovato")
    
        
#         # Calcola metriche finali
#     results['aggregated-terms'] = aggregate_scores_list
    
#     if aggregate_scores_list:
#         task_aggregate = harmonic_mean(aggregate_scores_list)
#         results['harmonic-mean-task-aggregate'] = task_aggregate
#         print(f"📈 Task aggregate (harmonic mean): {task_aggregate:.4f}")
#     else:
#         task_aggregate = -1
#         results['harmonic-mean-task-aggregate'] = -1
#         print("❌ Impossibile calcolare task aggregate")
    
#     results['aggregate-score'] = -1
    
#     # Calcola score finale se abbiamo MIA
#     if 'mia_loss_acc' in results and task_aggregate > 0:
#         mia_final_score = 1 - abs(results['mia_loss_acc'] - 0.5) * 2
#         results['mia_final_score'] = mia_final_score
#         results['aggregate-score'] = mean([task_aggregate, mia_final_score])
#         print(f"🎯 Final aggregate score: {results['aggregate-score']:.4f}")
    
#     # Salva risultati
#     metrics_file = os.path.join(OUTPUT_DIR, 'evaluation_results.json')
#     with open(metrics_file, 'w') as outptr:
#         json.dump(results, outptr, indent=2)
    
#     print(f"💾 Risultati salvati in: {metrics_file}")
    
#     # Stampa riepilogo
#     print("\n📋 RIEPILOGO RISULTATI:")
#     print("=" * 50)
    
#     for key, value in results.items():
#         if isinstance(value, dict):
#             print(f"{key}:")
#             for sub_key, sub_value in value.items():
#                 if isinstance(sub_value, float):
#                     print(f"  {sub_key}: {sub_value:.4f}")
#                 else:
#                     print(f"  {sub_key}: {sub_value}")
#         elif isinstance(value, (int, float)) and key != 'aggregated-terms':
#             print(f"{key}: {value:.4f}")
    
#     return results


# if __name__ == "__main__":
#     print("🚀 Avviando calcolo metriche...")
#     print(f"📁 Percorso risultati: {RESULTS_PATH}")
#     print(f"📁 Output directory: {OUTPUT_DIR}")
    
#     # Verifica che esistano i file necessari
#     if not os.path.exists(RESULTS_PATH):
#         print(f"❌ Path risultati non trovato: {RESULTS_PATH}")
#         exit(1)
    
    
#     # Esegui calcolo
#     try:
#         results = compute_metrics()
#         print("✅ Calcolo metriche completato con successo!")
#     except Exception as e:
#         print(f"❌ Errore durante il calcolo: {e}")
#         import traceback
#         traceback.print_exc()

🚀 Avviando calcolo metriche...
📁 Percorso risultati: /kaggle/input/eval-results
📁 Output directory: final_metrics
📊 Iniziando calcolo metriche...
🔍 Processando forget set...
   Trovati 1 file: ['/kaggle/input/eval-results/forget_0.csv']
   Caricato /kaggle/input/eval-results/forget_0.csv: 254 righe
   Dataset combinato: 254 righe
   Calcolando metriche per 254 esempi...
   Regurgitation score medio: 0.7811
   Knowledge score medio: 0.3526
   ✅ forget set completato
🔍 Processando retain set...
   Trovati 1 file: ['/kaggle/input/eval-results/retain_0.csv']
   Caricato /kaggle/input/eval-results/retain_0.csv: 278 righe
   Dataset combinato: 278 righe
   Calcolando metriche per 278 esempi...
   Regurgitation score medio: 0.7845
   Knowledge score medio: 0.3862
   ✅ retain set completato
🔍 Processando MIA data...
/kaggle/input/mia-score/member.jsonl
pattern
/kaggle/input/mia-score/member.jsonl
   Trovati file member: ['/kaggle/input/mia-score/member.jsonl']
❌ Errore durante il calcolo: name

Traceback (most recent call last):
  File "/tmp/ipykernel_36/4153794819.py", line 275, in <cell line: 0>
    results = compute_metrics()
              ^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_36/4153794819.py", line 201, in compute_metrics
    nll = compute_nll(text)         # <--- funzione che abbiamo definito prima
          ^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_36/4153794819.py", line 20, in compute_nll
    outputs = student_model(**enc, labels=enc["input_ids"])
              ^^^^^^^^^^^^^
NameError: name 'student_model' is not defined
